# ***CIFAR-10 Train AI Model***

## Modèle de base

In [ ]:
# -*- coding: utf-8 -*-
"""
VGG11 from scratch pour CIFAR-10 (32x32), simple — sans régularisation, sans callbacks
"""

import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

print(tf.__version__)

#############################################################################################################################

# Affiche si un GPU est détecté (TF choisit automatiquement)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU détecté :", gpus)
else:
    print("⚠️ Pas de GPU détecté — entraînement sur CPU.")

#############################################################################################################################

# Timer minimal
class timer:
    def __init__(self):
        self.start = None
        self.stop = None
    def tic(self):
        self.start = time.time()
    def toc(self):
        self.stop = time.time()
    def res(self):
        return None if self.start is None or self.stop is None else self.stop - self.start

#############################################################################################################################

def build_vgg11(input_shape):
    model = models.Sequential(name="VGG11_CIFAR10_simple")
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(10, activation='softmax'))

    return model

#############################################################################################################################

# Chargement et préparation des données CIFAR-10
class dataset:
    def __init__(self, nb_epochs=20, batch_size=64):
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

        x_train = x_train.astype('float32') / 255.0
        x_test  = x_test.astype('float32') / 255.0

        self.x_train = x_train
        self.y_train = y_train

        self.x_test = x_test
        self.y_test = y_test

        self.input_shape = self.x_train.shape[1:]
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size

        # one-hot
        self.y_train = tf.keras.utils.to_categorical(self.y_train, 10)
        self.y_test = tf.keras.utils.to_categorical(self.y_test, 10)

        print("Train:", self.x_train.shape, self.y_train.shape)
        print("Test: ", self.x_test.shape, self.y_test.shape)
        print("Epochs:", self.nb_epochs, " Batch size:", self.batch_size)

#############################################################################################################################

# Entraînement simple
def train_model(data):
    print("➡️ Construction du modèle...")
    model = build_vgg11(data.input_shape)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    t = timer()
    t.tic()
    history = model.fit(
        data.x_train, data.y_train,
        validation_data=(data.x_test, data.y_test),
        epochs=data.nb_epochs,
        batch_size=data.batch_size,
        shuffle=True,
        verbose=2
    )
    t.toc()
    print(f"✅ Entraînement terminé en {t.res():.1f} s")
    return model, history

#############################################################################################################################

# Évaluation
def test_model(data, model):
    loss, acc = model.evaluate(data.x_test, data.y_test, verbose=2)
    print(f"Test loss: {loss:.4f}  Test accuracy: {acc:.4f}")

# (Facultatif) affichage des courbes
def plot_history(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train loss'); plt.plot(history.history['val_loss'], label='val loss')
    plt.legend(); plt.title('Loss')
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='train acc'); plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

#############################################################################################################################

# MAIN
if __name__ == '__main__':
    # Hyperparamètres simples
    NB_EPOCHS = 50
    BATCH_SIZE = 64  # si VRAM limitée -> réduire à 32 ou 16

    data = dataset(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE)
    model, history = train_model(data)
    test_model(data, model)
    plot_history(history)

#############################################################################################################################

from google.colab import drive
drive.mount('/content/drive')
model.save("/content/drive/MyDrive/Colab Notebooks/CIFAR-models/CIFAR10_VGG11_modele_base.h5")

2.19.0
✅ GPU détecté : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Train: (50000, 32, 32, 3) (50000, 10)
Test:  (10000, 32, 32, 3) (10000, 10)
Epochs: 50  Batch size: 64
➡️ Construction du modèle...


Model: "VGG11_CIFAR10_simple"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_2             │ (None, 32, 32, 32)     │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_3             │ (None, 16, 16, 64)     │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_4             │ (None, 8, 8, 128)      │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 4, 4, 128)      │             

 Total params: 1,344,042 (5.13 MB)

 Trainable params: 1,343,146 (5.12 MB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/50


KeyboardInterrupt: 

## Modèle 1 - Couche vectorisation 1D

In [ ]:
# -*- coding: utf-8 -*-
"""
VGG11 from scratch pour CIFAR-10 (32x32), simple — sans régularisation, sans callbacks
"""

import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

print(tf.__version__)

#############################################################################################################################

# Affiche si un GPU est détecté (TF choisit automatiquement)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU détecté :", gpus)
else:
    print("⚠️ Pas de GPU détecté — entraînement sur CPU.")

#############################################################################################################################

# Timer minimal
class timer:
    def __init__(self):
        self.start = None
        self.stop = None
    def tic(self):
        self.start = time.time()
    def toc(self):
        self.stop = time.time()
    def res(self):
        return None if self.start is None or self.stop is None else self.stop - self.start

#############################################################################################################################

def build_vgg11(input_shape):
    model = models.Sequential(name="VGG11_CIFAR10_simple")
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(10, activation='softmax'))

    return model

#############################################################################################################################

# Chargement et préparation des données CIFAR-10
class dataset:
    def __init__(self, nb_epochs=20, batch_size=64):
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

        x_train = x_train.astype('float32') / 255.0
        x_test  = x_test.astype('float32') / 255.0

        self.x_train = x_train
        self.y_train = y_train

        self.x_test = x_test
        self.y_test = y_test

        self.input_shape = self.x_train.shape[1:]
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size

        # one-hot
        self.y_train = tf.keras.utils.to_categorical(self.y_train, 10)
        self.y_test = tf.keras.utils.to_categorical(self.y_test, 10)

        print("Train:", self.x_train.shape, self.y_train.shape)
        print("Test: ", self.x_test.shape, self.y_test.shape)
        print("Epochs:", self.nb_epochs, " Batch size:", self.batch_size)

#############################################################################################################################

# Entraînement simple
def train_model(data):
    print("➡️ Construction du modèle...")
    model = build_vgg11(data.input_shape)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    t = timer()
    t.tic()
    history = model.fit(
        data.x_train, data.y_train,
        validation_data=(data.x_test, data.y_test),
        epochs=data.nb_epochs,
        batch_size=data.batch_size,
        shuffle=True,
        verbose=2
    )
    t.toc()
    print(f"✅ Entraînement terminé en {t.res():.1f} s")
    return model, history

#############################################################################################################################

# Évaluation
def test_model(data, model):
    loss, acc = model.evaluate(data.x_test, data.y_test, verbose=2)
    print(f"Test loss: {loss:.4f}  Test accuracy: {acc:.4f}")

# (Facultatif) affichage des courbes
def plot_history(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train loss'); plt.plot(history.history['val_loss'], label='val loss')
    plt.legend(); plt.title('Loss')
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='train acc'); plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

#############################################################################################################################

# MAIN
if __name__ == '__main__':
    # Hyperparamètres simples
    NB_EPOCHS = 50
    BATCH_SIZE = 64  # si VRAM limitée -> réduire à 32 ou 16

    data = dataset(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE)
    model, history = train_model(data)
    test_model(data, model)
    plot_history(history)

#############################################################################################################################

from google.colab import drive
drive.mount('/content/drive')
model.save("/content/drive/MyDrive/Colab Notebooks/CIFAR-models/CIFAR10_VGG11_modele1.h5")

## Modèle 1-1 - Adaptation Dropout

In [ ]:
# -*- coding: utf-8 -*-
"""
VGG11 from scratch pour CIFAR-10 (32x32), simple — sans régularisation, sans callbacks
"""

import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

print(tf.__version__)

#############################################################################################################################

# Affiche si un GPU est détecté (TF choisit automatiquement)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU détecté :", gpus)
else:
    print("⚠️ Pas de GPU détecté — entraînement sur CPU.")

#############################################################################################################################

# Timer minimal
class timer:
    def __init__(self):
        self.start = None
        self.stop = None
    def tic(self):
        self.start = time.time()
    def toc(self):
        self.stop = time.time()
    def res(self):
        return None if self.start is None or self.stop is None else self.stop - self.start

#############################################################################################################################

def build_vgg11(input_shape):
    model = models.Sequential(name="VGG11_CIFAR10_simple")
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.15))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.15))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.15))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.15))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10, activation='softmax'))

    return model

#############################################################################################################################

# Chargement et préparation des données CIFAR-10
class dataset:
    def __init__(self, nb_epochs=20, batch_size=64):
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

        x_train = x_train.astype('float32') / 255.0
        x_test  = x_test.astype('float32') / 255.0

        self.x_train = x_train
        self.y_train = y_train

        self.x_test = x_test
        self.y_test = y_test

        self.input_shape = self.x_train.shape[1:]
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size

        # one-hot
        self.y_train = tf.keras.utils.to_categorical(self.y_train, 10)
        self.y_test = tf.keras.utils.to_categorical(self.y_test, 10)

        print("Train:", self.x_train.shape, self.y_train.shape)
        print("Test: ", self.x_test.shape, self.y_test.shape)
        print("Epochs:", self.nb_epochs, " Batch size:", self.batch_size)

#############################################################################################################################

# Entraînement simple
def train_model(data):
    print("➡️ Construction du modèle...")
    model = build_vgg11(data.input_shape)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    t = timer()
    t.tic()
    history = model.fit(
        data.x_train, data.y_train,
        validation_data=(data.x_test, data.y_test),
        epochs=data.nb_epochs,
        batch_size=data.batch_size,
        shuffle=True,
        verbose=2
    )
    t.toc()
    print(f"✅ Entraînement terminé en {t.res():.1f} s")
    return model, history

#############################################################################################################################

# Évaluation
def test_model(data, model):
    loss, acc = model.evaluate(data.x_test, data.y_test, verbose=2)
    print(f"Test loss: {loss:.4f}  Test accuracy: {acc:.4f}")

# (Facultatif) affichage des courbes
def plot_history(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train loss'); plt.plot(history.history['val_loss'], label='val loss')
    plt.legend(); plt.title('Loss')
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='train acc'); plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

#############################################################################################################################

# MAIN
if __name__ == '__main__':
    # Hyperparamètres simples
    NB_EPOCHS = 50
    BATCH_SIZE = 64  # si VRAM limitée -> réduire à 32 ou 16

    data = dataset(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE)
    model, history = train_model(data)
    test_model(data, model)
    plot_history(history)

#############################################################################################################################

from google.colab import drive
drive.mount('/content/drive')
model.save("/content/drive/MyDrive/Colab Notebooks/CIFAR-models/CIFAR10_VGG11_modele_1-1.h5")

## Modèle 2 - Suppression de couches

In [ ]:
# -*- coding: utf-8 -*-
"""
VGG11 from scratch pour CIFAR-10 (32x32), simple — sans régularisation, sans callbacks
"""

import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

print(tf.__version__)

#############################################################################################################################

# Affiche si un GPU est détecté (TF choisit automatiquement)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU détecté :", gpus)
else:
    print("⚠️ Pas de GPU détecté — entraînement sur CPU.")

#############################################################################################################################

# Timer minimal
class timer:
    def __init__(self):
        self.start = None
        self.stop = None
    def tic(self):
        self.start = time.time()
    def toc(self):
        self.stop = time.time()
    def res(self):
        return None if self.start is None or self.stop is None else self.stop - self.start

#############################################################################################################################

def build_vgg11(input_shape):
    model = models.Sequential(name="VGG11_CIFAR10_simple")
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.25))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(10, activation='softmax'))

    return model

#############################################################################################################################

# Chargement et préparation des données CIFAR-10
class dataset:
    def __init__(self, nb_epochs=20, batch_size=64):
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

        x_train = x_train.astype('float32') / 255.0
        x_test  = x_test.astype('float32') / 255.0

        self.x_train = x_train
        self.y_train = y_train

        self.x_test = x_test
        self.y_test = y_test

        self.input_shape = self.x_train.shape[1:]
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size

        # one-hot
        self.y_train = tf.keras.utils.to_categorical(self.y_train, 10)
        self.y_test = tf.keras.utils.to_categorical(self.y_test, 10)

        print("Train:", self.x_train.shape, self.y_train.shape)
        print("Test: ", self.x_test.shape, self.y_test.shape)
        print("Epochs:", self.nb_epochs, " Batch size:", self.batch_size)

#############################################################################################################################

# Entraînement simple
def train_model(data):
    print("➡️ Construction du modèle...")
    model = build_vgg11(data.input_shape)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    t = timer()
    t.tic()
    history = model.fit(
        data.x_train, data.y_train,
        validation_data=(data.x_test, data.y_test),
        epochs=data.nb_epochs,
        batch_size=data.batch_size,
        shuffle=True,
        verbose=2
    )
    t.toc()
    print(f"✅ Entraînement terminé en {t.res():.1f} s")
    return model, history

#############################################################################################################################

# Évaluation
def test_model(data, model):
    loss, acc = model.evaluate(data.x_test, data.y_test, verbose=2)
    print(f"Test loss: {loss:.4f}  Test accuracy: {acc:.4f}")

# (Facultatif) affichage des courbes
def plot_history(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train loss'); plt.plot(history.history['val_loss'], label='val loss')
    plt.legend(); plt.title('Loss')
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='train acc'); plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

#############################################################################################################################

# MAIN
if __name__ == '__main__':
    # Hyperparamètres simples
    NB_EPOCHS = 50
    BATCH_SIZE = 64  # si VRAM limitée -> réduire à 32 ou 16

    data = dataset(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE)
    model, history = train_model(data)
    test_model(data, model)
    plot_history(history)

#############################################################################################################################

from google.colab import drive
drive.mount('/content/drive')
model.save("/content/drive/MyDrive/Colab Notebooks/CIFAR-models/CIFAR10_VGG11_modele2.h5")

## Modèle 2-1 - Adaptation du Dropout

In [ ]:
# -*- coding: utf-8 -*-
"""
VGG11 from scratch pour CIFAR-10 (32x32), simple — sans régularisation, sans callbacks
"""

import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

print(tf.__version__)

#############################################################################################################################

# Affiche si un GPU est détecté (TF choisit automatiquement)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU détecté :", gpus)
else:
    print("⚠️ Pas de GPU détecté — entraînement sur CPU.")

#############################################################################################################################

# Timer minimal
class timer:
    def __init__(self):
        self.start = None
        self.stop = None
    def tic(self):
        self.start = time.time()
    def toc(self):
        self.stop = time.time()
    def res(self):
        return None if self.start is None or self.stop is None else self.stop - self.start

#############################################################################################################################

def build_vgg11(input_shape):
    model = models.Sequential(name="VGG11_CIFAR10_simple")
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10, activation='softmax'))

    return model

#############################################################################################################################

# Chargement et préparation des données CIFAR-10
class dataset:
    def __init__(self, nb_epochs=20, batch_size=64):
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

        x_train = x_train.astype('float32') / 255.0
        x_test  = x_test.astype('float32') / 255.0

        self.x_train = x_train
        self.y_train = y_train

        self.x_test = x_test
        self.y_test = y_test

        self.input_shape = self.x_train.shape[1:]
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size

        # one-hot
        self.y_train = tf.keras.utils.to_categorical(self.y_train, 10)
        self.y_test = tf.keras.utils.to_categorical(self.y_test, 10)

        print("Train:", self.x_train.shape, self.y_train.shape)
        print("Test: ", self.x_test.shape, self.y_test.shape)
        print("Epochs:", self.nb_epochs, " Batch size:", self.batch_size)

#############################################################################################################################

# Entraînement simple
def train_model(data):
    print("➡️ Construction du modèle...")
    model = build_vgg11(data.input_shape)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    t = timer()
    t.tic()
    history = model.fit(
        data.x_train, data.y_train,
        validation_data=(data.x_test, data.y_test),
        epochs=data.nb_epochs,
        batch_size=data.batch_size,
        shuffle=True,
        verbose=2
    )
    t.toc()
    print(f"✅ Entraînement terminé en {t.res():.1f} s")
    return model, history

#############################################################################################################################

# Évaluation
def test_model(data, model):
    loss, acc = model.evaluate(data.x_test, data.y_test, verbose=2)
    print(f"Test loss: {loss:.4f}  Test accuracy: {acc:.4f}")

# (Facultatif) affichage des courbes
def plot_history(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train loss'); plt.plot(history.history['val_loss'], label='val loss')
    plt.legend(); plt.title('Loss')
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='train acc'); plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

#############################################################################################################################

# MAIN
if __name__ == '__main__':
    # Hyperparamètres simples
    NB_EPOCHS = 50
    BATCH_SIZE = 64  # si VRAM limitée -> réduire à 32 ou 16

    data = dataset(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE)
    model, history = train_model(data)
    test_model(data, model)
    plot_history(history)

#############################################################################################################################

from google.colab import drive
drive.mount('/content/drive')
model.save("/content/drive/MyDrive/Colab Notebooks/CIFAR-models/CIFAR10_VGG11_modele2-1.h5")

## Modèle 2-2 - Ajout d'une phase de MaxPooling

In [ ]:
# -*- coding: utf-8 -*-
"""
VGG11 from scratch pour CIFAR-10 (32x32), simple — sans régularisation, sans callbacks
"""

import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

print(tf.__version__)

#############################################################################################################################

# Affiche si un GPU est détecté (TF choisit automatiquement)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU détecté :", gpus)
else:
    print("⚠️ Pas de GPU détecté — entraînement sur CPU.")

#############################################################################################################################

# Timer minimal
class timer:
    def __init__(self):
        self.start = None
        self.stop = None
    def tic(self):
        self.start = time.time()
    def toc(self):
        self.stop = time.time()
    def res(self):
        return None if self.start is None or self.stop is None else self.stop - self.start

#############################################################################################################################

def build_vgg11(input_shape):
    model = models.Sequential(name="VGG11_CIFAR10_simple")
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.15))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.15))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.15))
    model.add(layers.Dense(10, activation='softmax'))

    return model

#############################################################################################################################

# Chargement et préparation des données CIFAR-10
class dataset:
    def __init__(self, nb_epochs=20, batch_size=64):
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

        x_train = x_train.astype('float32') / 255.0
        x_test  = x_test.astype('float32') / 255.0

        self.x_train = x_train
        self.y_train = y_train

        self.x_test = x_test
        self.y_test = y_test

        self.input_shape = self.x_train.shape[1:]
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size

        # one-hot
        self.y_train = tf.keras.utils.to_categorical(self.y_train, 10)
        self.y_test = tf.keras.utils.to_categorical(self.y_test, 10)

        print("Train:", self.x_train.shape, self.y_train.shape)
        print("Test: ", self.x_test.shape, self.y_test.shape)
        print("Epochs:", self.nb_epochs, " Batch size:", self.batch_size)

#############################################################################################################################

# Entraînement simple
def train_model(data):
    print("➡️ Construction du modèle...")
    model = build_vgg11(data.input_shape)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    t = timer()
    t.tic()
    history = model.fit(
        data.x_train, data.y_train,
        validation_data=(data.x_test, data.y_test),
        epochs=data.nb_epochs,
        batch_size=data.batch_size,
        shuffle=True,
        verbose=2
    )
    t.toc()
    print(f"✅ Entraînement terminé en {t.res():.1f} s")
    return model, history

#############################################################################################################################

# Évaluation
def test_model(data, model):
    loss, acc = model.evaluate(data.x_test, data.y_test, verbose=2)
    print(f"Test loss: {loss:.4f}  Test accuracy: {acc:.4f}")

# (Facultatif) affichage des courbes
def plot_history(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train loss'); plt.plot(history.history['val_loss'], label='val loss')
    plt.legend(); plt.title('Loss')
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='train acc'); plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

#############################################################################################################################

# MAIN
if __name__ == '__main__':
    # Hyperparamètres simples
    NB_EPOCHS = 50
    BATCH_SIZE = 64  # si VRAM limitée -> réduire à 32 ou 16

    data = dataset(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE)
    model, history = train_model(data)
    test_model(data, model)
    plot_history(history)

#############################################################################################################################

from google.colab import drive
drive.mount('/content/drive')
model.save("/content/drive/MyDrive/Colab Notebooks/CIFAR-models/CIFAR10_VGG11_modele2-2.h5")

## Modèle 3 - Variation du Learning Rate

In [ ]:
# -*- coding: utf-8 -*-
"""
VGG11 from scratch pour CIFAR-10 (32x32), simple — sans régularisation, sans callbacks
"""

import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

print(tf.__version__)

#############################################################################################################################

# Affiche si un GPU est détecté (TF choisit automatiquement)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU détecté :", gpus)
else:
    print("⚠️ Pas de GPU détecté — entraînement sur CPU.")

#############################################################################################################################

# Timer minimal
class timer:
    def __init__(self):
        self.start = None
        self.stop = None
    def tic(self):
        self.start = time.time()
    def toc(self):
        self.stop = time.time()
    def res(self):
        return None if self.start is None or self.stop is None else self.stop - self.start

#############################################################################################################################

def build_vgg11(input_shape):
    model = models.Sequential(name="VGG11_CIFAR10_simple")
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv2D(16, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(16, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.11))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(32, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.11))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(10, activation='softmax'))

    return model

#############################################################################################################################

# Chargement et préparation des données CIFAR-10
class dataset:
    def __init__(self, nb_epochs=20, batch_size=64):
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

        x_train = x_train.astype('float32') / 255.0
        x_test  = x_test.astype('float32') / 255.0

        self.x_train = x_train
        self.y_train = y_train

        self.x_test = x_test
        self.y_test = y_test

        self.input_shape = self.x_train.shape[1:]
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size

        # one-hot
        self.y_train = tf.keras.utils.to_categorical(self.y_train, 10)
        self.y_test = tf.keras.utils.to_categorical(self.y_test, 10)

        print("Train:", self.x_train.shape, self.y_train.shape)
        print("Test: ", self.x_test.shape, self.y_test.shape)
        print("Epochs:", self.nb_epochs, " Batch size:", self.batch_size)

#############################################################################################################################

# Entraînement simple
def train_model(data):
    print("➡️ Construction du modèle...")
    model = build_vgg11(data.input_shape)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    initial_lr = 0.01
    warmup_epochs = 5
    total_epochs = data.nb_epochs

    def lr_schedule(epoch):
        if epoch < warmup_epochs:
            return float(initial_lr * (epoch + 1) / warmup_epochs)
        else:
            progress = (epoch - warmup_epochs) / (total_epochs - warmup_epochs)
            return float(0.5 * initial_lr * (1 + tf.math.cos(tf.constant(progress * 3.1416))))

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    t = timer()
    t.tic()
    history = model.fit(
        data.x_train, data.y_train,
        validation_data=(data.x_test, data.y_test),
        epochs=data.nb_epochs,
        batch_size=data.batch_size,
        shuffle=True,
        callbacks=[lr_callback],
        verbose=2
    )
    t.toc()
    print(f"✅ Entraînement terminé en {t.res():.1f} s")
    return model, history

#############################################################################################################################

# Évaluation
def test_model(data, model):
    loss, acc = model.evaluate(data.x_test, data.y_test, verbose=2)
    print(f"Test loss: {loss:.4f}  Test accuracy: {acc:.4f}")

# (Facultatif) affichage des courbes
def plot_history(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train loss'); plt.plot(history.history['val_loss'], label='val loss')
    plt.legend(); plt.title('Loss')
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='train acc'); plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

#############################################################################################################################

# MAIN
if __name__ == '__main__':
    # Hyperparamètres simples
    NB_EPOCHS = 50
    BATCH_SIZE = 64  # si VRAM limitée -> réduire à 32 ou 16

    data = dataset(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE)
    model, history = train_model(data)
    test_model(data, model)
    plot_history(history)

#############################################################################################################################

from google.colab import drive
drive.mount('/content/drive')
model.save("/content/drive/MyDrive/Colab Notebooks/CIFAR-models/CIFAR10_VGG11_modele3.h5")

## Modèle 3-1 - Paramètrage de la Loss

In [ ]:
# -*- coding: utf-8 -*-
"""
VGG11 from scratch pour CIFAR-10 (32x32), simple — sans régularisation, sans callbacks
"""

import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

print(tf.__version__)

#############################################################################################################################

# Affiche si un GPU est détecté (TF choisit automatiquement)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU détecté :", gpus)
else:
    print("⚠️ Pas de GPU détecté — entraînement sur CPU.")

#############################################################################################################################

# Timer minimal
class timer:
    def __init__(self):
        self.start = None
        self.stop = None
    def tic(self):
        self.start = time.time()
    def toc(self):
        self.stop = time.time()
    def res(self):
        return None if self.start is None or self.stop is None else self.stop - self.start

#############################################################################################################################

def build_vgg11(input_shape):
    model = models.Sequential(name="VGG11_CIFAR10_simple")
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv2D(16, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(16, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.05))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(16, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(16, (3,3), padding='same', use_bias=True))
    model.add(layers.Activation('relu'))
    model.add(layers.SpatialDropout2D(0.05))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.06))
    model.add(layers.Dense(10, activation='softmax'))

    return model

#############################################################################################################################

# Chargement et préparation des données CIFAR-10
class dataset:
    def __init__(self, nb_epochs=20, batch_size=64):
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

        x_train = x_train.astype('float32') / 255.0
        x_test  = x_test.astype('float32') / 255.0

        self.x_train = x_train
        self.y_train = y_train

        self.x_test = x_test
        self.y_test = y_test

        self.input_shape = self.x_train.shape[1:]
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size

        # one-hot
        self.y_train = tf.keras.utils.to_categorical(self.y_train, 10)
        self.y_test = tf.keras.utils.to_categorical(self.y_test, 10)

        print("Train:", self.x_train.shape, self.y_train.shape)
        print("Test: ", self.x_test.shape, self.y_test.shape)
        print("Epochs:", self.nb_epochs, " Batch size:", self.batch_size)

#############################################################################################################################

# Entraînement simple
def train_model(data):
    print("➡️ Construction du modèle...")
    model = build_vgg11(data.input_shape)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1), metrics=['accuracy'])
    model.summary()

    initial_lr = 0.01
    warmup_epochs = 5
    total_epochs = data.nb_epochs

    def lr_schedule(epoch):
        if epoch < warmup_epochs:
            return float(initial_lr * (epoch + 1) / warmup_epochs)
        else:
            progress = (epoch - warmup_epochs) / (total_epochs - warmup_epochs)
            return float(0.5 * initial_lr * (1 + tf.math.cos(tf.constant(progress * 3.1416))))

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    t = timer()
    t.tic()
    history = model.fit(
        data.x_train, data.y_train,
        validation_data=(data.x_test, data.y_test),
        epochs=data.nb_epochs,
        batch_size=data.batch_size,
        shuffle=True,
        callbacks=[lr_callback],
        verbose=2
    )
    t.toc()
    print(f"✅ Entraînement terminé en {t.res():.1f} s")
    return model, history

#############################################################################################################################

# Évaluation
def test_model(data, model):
    loss, acc = model.evaluate(data.x_test, data.y_test, verbose=2)
    print(f"Test loss: {loss:.4f}  Test accuracy: {acc:.4f}")

# (Facultatif) affichage des courbes
def plot_history(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train loss'); plt.plot(history.history['val_loss'], label='val loss')
    plt.legend(); plt.title('Loss')
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='train acc'); plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

#############################################################################################################################

# MAIN
if __name__ == '__main__':
    # Hyperparamètres simples
    NB_EPOCHS = 50
    BATCH_SIZE = 64  # si VRAM limitée -> réduire à 32 ou 16

    data = dataset(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE)
    model, history = train_model(data)
    test_model(data, model)
    plot_history(history)

#############################################################################################################################

from google.colab import drive
drive.mount('/content/drive')
model.save("/content/drive/MyDrive/Colab Notebooks/CIFAR-models/CIFAR10_VGG11_modele3-1.h5")

2.19.0
✅ GPU détecté : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Train: (50000, 32, 32, 3) (50000, 10)
Test:  (10000, 32, 32, 3) (10000, 10)
Epochs: 50  Batch size: 64
➡️ Construction du modèle...


Model: "VGG11_CIFAR10_simple"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 32, 32, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 32, 32, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 32, 32, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_6             │ (None, 32, 32, 16)     │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 32, 32, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 16, 16, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 16, 16, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 8, 8, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 8, 8, 16)       │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 8, 8, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_7             │ (None, 8, 8, 16)       │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 8, 8, 16)       │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 4, 4, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 16)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │         2,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴─────────────

 Total params: 11,130 (43.48 KB)

 Trainable params: 11,002 (42.98 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/50
782/782 - 19s - 24ms/step - accuracy: 0.4191 - loss: 1.7203 - val_accuracy: 0.4353 - val_loss: 1.7375 - learning_rate: 0.0020
Epoch 2/50
782/782 - 9s - 12ms/step - accuracy: 0.5200 - loss: 1.5395 - val_accuracy: 0.5216 - val_loss: 1.5373 - learning_rate: 0.0040
Epoch 3/50
782/782 - 5s - 6ms/step - accuracy: 0.5539 - loss: 1.4772 - val_accuracy: 0.5086 - val_loss: 1.5751 - learning_rate: 0.0060
Epoch 4/50
782/782 - 3s - 4ms/step - accuracy: 0.5735 - loss: 1.4460 - val_accuracy: 0.5377 - val_loss: 1.5303 - learning_rate: 0.0080
Epoch 5/50
782/782 - 4s - 5ms/step - accuracy: 0.5815 - loss: 1.4279 - val_accuracy: 0.5169 - val_loss: 1.5230 - learning_rate: 0.0100
Epoch 6/50
782/782 - 4s - 5ms/step - accuracy: 0.5976 - loss: 1.3947 - val_accuracy: 0.5479 - val_loss: 1.5023 - learning_rate: 0.0100
Epoch 7/50
782/782 - 3s - 4ms/step - accuracy: 0.6111 - loss: 1.3679 - val_accuracy: 0.5884 - val_loss: 1.4199 - learning_rate: 0.0100
Epoch 8/50
782/782 - 4s - 5ms/step - accuracy: 0.617